 This model based on the architecture mentioned here https://docs.google.com/document/d/1iWhJiMT9pgWqYA_3-iRyvQ1DwlhV3hGdR-pinZiiHfk/edit


In [ ]:
from __future__ import division, print_function, absolute_import

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

import keras

import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot


define parameters

In [ ]:

# Load your Dataset
trainImg = pd.read_csv("/Your/dataset/training/file/in/csv/form", header=None)
trainLabel = pd.read_csv("/Your/dataset/training/labels/in/csv/form", header=None)
testImg = pd.read_csv("/Your/dataset/teastinf/file/in/csv/form", header=None)
testLabel = pd.read_csv("/Your/dataset/testing/labels/in/csv/form", header=None)


In [ ]:
trainImg.head()

In [ ]:
testImg.head()

In [ ]:
#training images
trainImg = trainImg.values.astype('float32') /255.0
#training labels
trainLabel = trainLabel.values.astype('int32') 

#testing images
testImg = testImg.values.astype('int32')/255.0
#testing labels
testLabel = testLabel.values.astype('float32')

In [ ]:
trainImg[0]

In [ ]:
#One Hot encoding of train labels.
trainLabel = to_categorical(trainLabel,10)

#One Hot encoding of test labels.
testLabel = to_categorical(testLabel,10)


In [ ]:
trainLabel[0]

In [ ]:
print(trainImg.shape, trainLabel.shape, testImg.shape, testLabel.shape)

In [ ]:
# reshape input images to 28x28x1
trainImg = trainImg.reshape([-1, 28, 28, 1])
testImg = testImg.reshape([-1, 28, 28, 1])

In [ ]:
print(trainImg.shape, trainLabel.shape, testImg.shape, testLabel.shape)

In [ ]:


# reshape input images to 28x28x1
trainImg = trainImg.reshape([-1, 28, 28, 1])
testImg = testImg.reshape([-1, 28, 28, 1])



In [ ]:
trainImg[0]

In [ ]:
def vgg(input_shape, n_classes):
  
  input = Input(input_shape)
  
  x = Conv2D(64, 3, padding='same', activation='relu')(input)
  x = Conv2D(64, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(128, 3, padding='same', activation='relu')(x)
  x = Conv2D(128, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(256, 3, padding='same', activation='relu')(x)
  x = Conv2D(256, 3, padding='same', activation='relu')(x)
  x = Conv2D(256, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)
  
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model

In [ ]:
inputShape = 28,28,1
nOfClasses=10
K.clear_session()
model = vgg(inputShape, nOfClasses)
model.summary()

In [ ]:
 SVG(model_to_dot(model).create(prog='dot', format='svg'))


In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

In [ ]:
model.fit(trainImg, trainLabel,batch_size=100, epochs=5, verbose=1)


In [ ]:
# Evaluate model
score = model.evaluate(testImg, testLabel)
print('loss : %0.2f%%' % (score[0] * 100))
print('test accuarcy: %0.2f%%' % (score[1] * 100))
model.save('Models/ModelName')